In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
mg = RasterModelGrid((5,10),500)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
#d_h = mg.add_ones('link', 'hydraulic__diameter')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
h = mg.add_zeros('node', 'hydraulic__head')
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

#add input to node 310
moulin_Q=1.
mg.at_node['input__discharge'][12]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
Q[mg.active_links] = 0.1*np.random.rand(mg.number_of_active_links)
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print("Number of links = ", mg.number_of_links)
print("Number of nodes = ", mg.number_of_nodes)
print("Number of active links = ", mg.number_of_active_links)
print("Number of core nodes = ", mg.number_of_core_nodes)

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)




#code added for fsolve algorithm
#dhdx = mg.add_zeros('link', 'hydraulic_head_gradient', noclobber=False)
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)


pfn = PresFlowNetwork(mg)
pfn.run_one_step()
mc = MeltCreep(mg, dt=250.)

Number of links =  85
Number of nodes =  50
Number of active links =  40
Number of core nodes =  24
Number of iterations = 1 tolerance = 0.943568520736  iteration time= 0.007875680923461914
Number of iterations = 2 tolerance = 0.395637343143  iteration time= 0.007472515106201172
Number of iterations = 3 tolerance = 0.195385062608  iteration time= 0.006991147994995117
Number of iterations = 4 tolerance = 0.0919334405312  iteration time= 0.007621049880981445
Number of iterations = 5 tolerance = 0.0374495846789  iteration time= 0.0073833465576171875
Number of iterations = 6 tolerance = 0.00930073289427  iteration time= 0.00850367546081543
Number of iterations = 7 tolerance = 0.000654376717649  iteration time= 0.006951093673706055
Number of iterations = 8 tolerance = 0.000179491113853  iteration time= 0.007075786590576172
Number of iterations = 9 tolerance = 4.61122127517e-05  iteration time= 0.006888628005981445
Number of iterations = 10 tolerance = 3.78381141604e-06  iteration time= 0.00

/home/mcoving/Data/github/landlab/landlab/components/conduit_networks/pressurized_flow.py:155: RuntimeWarning: divide by zero encountered in true_divide
  self.r = 8*f*L/(g*r_s**2.*d_h**5.)


In [14]:
#Version of function that uses matplotlib to create plots rather than PIL
#def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='viridis', magnitude= False, lw=5,x_name='X', y_name='Y', var_name=''):#, logvalues=False ):
#    link_head_x = grid.node_x[grid.node_at_link_head]
#    link_head_y = grid.node_y[grid.node_at_link_head]
#    link_tail_x = grid.node_x[grid.node_at_link_tail]
#    link_tail_y = grid.node_y[grid.node_at_link_tail]       
#    if magnitude:
#        values = abs(grid.at_link[value_name])
#    else:
#        values = grid.at_link[value_name]
#    #if logvalues:
#    #    values = np.log10(abs(grid.at_link[value_name]))
#        
#    #Normalize color values
#    if autoscale:
#        cnorm = colors.Normalize()
#        cnorm.autoscale(values)
#    else:
#        cnorm = colors.Normalize(vmin,vmax)
#    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
#    scalarMap.set_array(values)
#    #set_cmap()
#    for i, value in enumerate(values):
#    #    draw.line( ((link_head_x[i]*pixel_xscaling_factor,link_head_y[i]*pixel_yscaling_factor),(link_tail_x[i]*pixel_xscaling_factor,link_tail_y[i]*pixel_yscaling_factor)),fill=scalarMap.to_rgba(value,bytes=True),width=lw) 
#        xs = [link_head_x[i],link_tail_x[i]]
#        ys = [link_head_y[i],link_tail_y[i]]
#        img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
#    #imshow(np.asarray(im), origin='lower', extent=(0,grid.number_of_cell_columns,0,grid.number_of_cell_rows))
#    cb = colorbar(scalarMap)
#    cb.ax.set_ylabel(var_name)
#    xlabel(x_name)
#    ylabel(y_name)

In [7]:
from PIL import Image
from PIL import ImageDraw

def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='viridis', magnitude= False, lw=5, pixels_per_node=15, x_name='X', y_name='Y', var_name='', use_PIL=True):#, logvalues=False ):
    if use_PIL:
        pixel_xscaling_factor = pixels_per_node/grid.dx
        pixel_yscaling_factor = pixels_per_node/grid.dy
        im = Image.new('RGBA', ( int(round(grid.number_of_cell_columns*pixel_xscaling_factor)), int(round(grid.number_of_cell_rows*pixel_yscaling_factor)) ), (255,255,255,255))
        draw = ImageDraw.Draw(im)

    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]       
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    #if logvalues:
    #    values = np.log10(abs(grid.at_link[value_name]))
    
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    if use_PIL:
        for i, value in enumerate(values):
            draw.line( ((link_head_x[i]*pixel_xscaling_factor,link_head_y[i]*pixel_yscaling_factor),(link_tail_x[i]*pixel_xscaling_factor,link_tail_y[i]*pixel_yscaling_factor)),fill=scalarMap.to_rgba(value,bytes=True),width=lw) 
        imshow(np.asarray(im), origin='lower', extent=(0,grid.number_of_cell_columns,0,grid.number_of_cell_rows))
    else:
        for i, value in enumerate(values):
            xs = [link_head_x[i],link_tail_x[i]]
            ys = [link_head_y[i],link_tail_y[i]]
            img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))    
    cb = colorbar(scalarMap)
    cb.ax.set_ylabel(var_name)
    xlabel(x_name)
    ylabel(y_name)


In [15]:
nsteps = 10000
step_start = 1
fig1, axs = subplots(2,2,figsize=(10,6))
every = 50
for step in arange(nsteps)+step_start:
    print("step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean())
    pfn.run_one_step()    
    #mg.calc_grad_at_link(h, out=dhdx)
    #h = fsolve(network_residuals, h[mg.core_nodes])
    #Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(2,2,1)
        imshow_grid_at_node(mg, h, var_name='Head ($m$)', cmap='viridis',vmin=300,vmax=400)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        #ylim([300,400])
        subplot(2,2,2)
        plot_links(mg, 'conduit__discharge', magnitude=True, var_name='Discharge ($m^3/s$)', vmin=0., vmax=1, autoscale=False)
        #plot(mg.at_link['conduit__discharge'][mg.active_links])
        #ylabel('')
        #ylim([0,1.5])
        subplot(2,2,3)
        plot_links(mg, 'hydraulic__diameter', var_name='Hydraulic diameter ($m$)', autoscale=False, vmin=0, vmax=1.25)
        subplot(2,2,4)#, position=[0.2,0.2,0.9,0.9])
        plot(mg.at_link['hydraulic__diameter'][mg.active_links])
        xlabel('Link number')
        ylabel('Hydraulic diameter ($m$)')
        ylim(0,1.25)
        #ylim([0.5,1.5])
        image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
        #tight_layout()
        subplots_adjust(wspace=0.3)
        savefig(image_name)
        fig1.clf()

#im_ani = animation.ArtistAnimation(fig1,ims)

step = 1  avg d_h= 0.969209477525
Number of iterations = 1 tolerance = 2.48831390264e-05  iteration time= 0.008456707000732422
Number of iterations = 2 tolerance = 3.36399535357e-07  iteration time= 0.007994413375854492
max ddh =  1.23648145829e-05
step = 2  avg d_h= 0.96915573806
Number of iterations = 1 tolerance = 2.50235183181e-05  iteration time= 0.00801992416381836
Number of iterations = 2 tolerance = 3.08709244279e-09  iteration time= 0.007092714309692383
max ddh =  1.23645108692e-05
step = 3  avg d_h= 0.969102001776
Number of iterations = 1 tolerance = 2.50907690461e-05  iteration time= 0.007660865783691406
Number of iterations = 2 tolerance = 2.20689139407e-07  iteration time= 0.0073316097259521484
max ddh =  1.23643103265e-05
step = 4  avg d_h= 0.969048268725
Number of iterations = 1 tolerance = 2.48645254413e-05  iteration time= 0.007058620452880859
Number of iterations = 2 tolerance = 2.94405858164e-08  iteration time= 0.00754857063293457
max ddh =  1.23639036373e-05
step =

Number of iterations = 1 tolerance = 2.4782960336e-05  iteration time= 0.0076143741607666016
Number of iterations = 2 tolerance = 1.50535038879e-07  iteration time= 0.0077173709869384766
max ddh =  1.23534808647e-05
step = 40  avg d_h= 0.967115997296
Number of iterations = 1 tolerance = 2.50941494589e-05  iteration time= 0.008200883865356445
Number of iterations = 2 tolerance = 3.73046728369e-08  iteration time= 0.008984088897705078
max ddh =  1.23531739416e-05
step = 41  avg d_h= 0.967062381891
Number of iterations = 1 tolerance = 2.51098543942e-05  iteration time= 0.008884906768798828
Number of iterations = 2 tolerance = 1.70129726519e-07  iteration time= 0.007372617721557617
max ddh =  1.23528937928e-05
step = 42  avg d_h= 0.967008769672
Number of iterations = 1 tolerance = 2.51524794536e-05  iteration time= 0.00804758071899414
Number of iterations = 2 tolerance = 6.27812350162e-08  iteration time= 0.006936073303222656
max ddh =  1.23525821809e-05
step = 43  avg d_h= 0.966955160623


Number of iterations = 1 tolerance = 2.52490690953e-05  iteration time= 0.01164555549621582
Number of iterations = 2 tolerance = 4.01583951787e-08  iteration time= 0.014020204544067383
max ddh =  1.23422704315e-05
step = 77  avg d_h= 0.965134339806
Number of iterations = 1 tolerance = 2.48179665069e-05  iteration time= 0.009498357772827148
Number of iterations = 2 tolerance = 1.71331391795e-07  iteration time= 0.008258581161499023
max ddh =  1.23418435747e-05
step = 78  avg d_h= 0.965080841662
Number of iterations = 1 tolerance = 2.50489686204e-05  iteration time= 0.009874105453491211
Number of iterations = 2 tolerance = 1.71631953669e-07  iteration time= 0.009553194046020508
max ddh =  1.23415913524e-05
step = 79  avg d_h= 0.96502734671
Number of iterations = 1 tolerance = 2.50761394713e-05  iteration time= 0.0077474117279052734
Number of iterations = 2 tolerance = 5.58550023316e-08  iteration time= 0.01271963119506836
max ddh =  1.23412607589e-05
step = 80  avg d_h= 0.964973854912
Nu

Number of iterations = 2 tolerance = 1.01158862342e-07  iteration time= 0.0078029632568359375
max ddh =  1.23306604045e-05
step = 114  avg d_h= 0.963157013958
Number of iterations = 1 tolerance = 2.48910630742e-05  iteration time= 0.0075836181640625
Number of iterations = 2 tolerance = 3.07486948264e-08  iteration time= 0.008340597152709961
max ddh =  1.23302845199e-05
step = 115  avg d_h= 0.963103632673
Number of iterations = 1 tolerance = 2.49267264114e-05  iteration time= 0.007566213607788086
Number of iterations = 2 tolerance = 5.72096540974e-09  iteration time= 0.009060144424438477
max ddh =  1.23299555908e-05
step = 116  avg d_h= 0.963050254534
Number of iterations = 1 tolerance = 2.50460359746e-05  iteration time= 0.007292270660400391
Number of iterations = 2 tolerance = 2.2978253883e-08  iteration time= 0.008040428161621094
max ddh =  1.23296589088e-05
step = 117  avg d_h= 0.962996879557
Number of iterations = 1 tolerance = 2.50027307184e-05  iteration time= 0.00703549385070800

step = 151  avg d_h= 0.961184003729
Number of iterations = 1 tolerance = 2.50796032543e-05  iteration time= 0.009774446487426758
Number of iterations = 2 tolerance = 1.7464619714e-07  iteration time= 0.00927877426147461
max ddh =  1.23184766852e-05
step = 152  avg d_h= 0.961130738888
Number of iterations = 1 tolerance = 2.47359120817e-05  iteration time= 0.009642839431762695
Number of iterations = 2 tolerance = 8.45502129421e-08  iteration time= 0.008098125457763672
max ddh =  1.23181031896e-05
step = 153  avg d_h= 0.961077477164
Number of iterations = 1 tolerance = 2.48391572708e-05  iteration time= 0.00911092758178711
Number of iterations = 2 tolerance = 1.74544120043e-07  iteration time= 0.008498430252075195
max ddh =  1.23177800413e-05
step = 154  avg d_h= 0.961024218581
Number of iterations = 1 tolerance = 2.51563904543e-05  iteration time= 0.0074574947357177734
Number of iterations = 2 tolerance = 8.92722676199e-08  iteration time= 0.0074999332427978516
max ddh =  1.23174581126e-

...

Number of iterations = 2 tolerance = 4.2564543782e-09  iteration time= 0.008207559585571289
max ddh =  7.3877709743e-06
step = 9826  avg d_h= 0.563168052974
Number of iterations = 1 tolerance = 1.49725766769e-05  iteration time= 0.008367776870727539
Number of iterations = 2 tolerance = 4.25530599386e-09  iteration time= 0.013370990753173828
max ddh =  7.3871114627e-06
step = 9827  avg d_h= 0.563136986746
Number of iterations = 1 tolerance = 1.49714344169e-05  iteration time= 0.009051322937011719
Number of iterations = 2 tolerance = 1.54294649279e-09  iteration time= 0.00751185417175293
max ddh =  7.38645164368e-06
step = 9828  avg d_h= 0.563105922285
Number of iterations = 1 tolerance = 1.49748676128e-05  iteration time= 0.0076291561126708984
Number of iterations = 2 tolerance = 5.47215595059e-09  iteration time= 0.006973981857299805
max ddh =  7.38579492169e-06
step = 9829  avg d_h= 0.56307485959
Number of iterations = 1 tolerance = 1.49581931622e-05  iteration time= 0.007991790771484

Number of iterations = 2 tolerance = 3.56268581303e-09  iteration time= 0.007749795913696289
max ddh =  7.36271609933e-06
step = 9864  avg d_h= 0.561988777455
Number of iterations = 1 tolerance = 1.49193843023e-05  iteration time= 0.008078336715698242
Number of iterations = 2 tolerance = 8.45124537192e-09  iteration time= 0.009251594543457031
max ddh =  7.36205704293e-06
step = 9865  avg d_h= 0.561957778297
Number of iterations = 1 tolerance = 1.49181248335e-05  iteration time= 0.007109165191650391
Number of iterations = 2 tolerance = 4.18618423433e-09  iteration time= 0.007037639617919922
max ddh =  7.36139419624e-06
step = 9866  avg d_h= 0.561926780901
Number of iterations = 1 tolerance = 1.49333694384e-05  iteration time= 0.007768154144287109
Number of iterations = 2 tolerance = 2.35952823946e-09  iteration time= 0.007250547409057617
max ddh =  7.36073781642e-06
step = 9867  avg d_h= 0.561895785269
Number of iterations = 1 tolerance = 1.49186405561e-05  iteration time= 0.00757980346

step = 9901  avg d_h= 0.560842982398
Number of iterations = 1 tolerance = 1.4878289665e-05  iteration time= 0.007789134979248047
Number of iterations = 2 tolerance = 4.70207691186e-09  iteration time= 0.006971597671508789
max ddh =  7.33768269945e-06
step = 9902  avg d_h= 0.560812048433
Number of iterations = 1 tolerance = 1.48754113952e-05  iteration time= 0.008005857467651367
Number of iterations = 2 tolerance = 2.33618259113e-09  iteration time= 0.006849050521850586
max ddh =  7.33702460835e-06
step = 9903  avg d_h= 0.560781116228
Number of iterations = 1 tolerance = 1.48641863323e-05  iteration time= 0.007987260818481445
Number of iterations = 2 tolerance = 8.31481115644e-09  iteration time= 0.007107973098754883
max ddh =  7.33636510238e-06
step = 9904  avg d_h= 0.560750185783
Number of iterations = 1 tolerance = 1.48690038647e-05  iteration time= 0.008895158767700195
Number of iterations = 2 tolerance = 5.60449758824e-09  iteration time= 0.00739598274230957
max ddh =  7.3357084965

Number of iterations = 2 tolerance = 5.76577030564e-09  iteration time= 0.0084075927734375
max ddh =  7.31333610198e-06
step = 9939  avg d_h= 0.559668728654
Number of iterations = 1 tolerance = 1.48207081796e-05  iteration time= 0.0072765350341796875
Number of iterations = 2 tolerance = 1.27311348963e-09  iteration time= 0.009345293045043945
max ddh =  7.3126756649e-06
step = 9940  avg d_h= 0.559637861531
Number of iterations = 1 tolerance = 1.48202566296e-05  iteration time= 0.007584571838378906
Number of iterations = 2 tolerance = 8.18500753893e-09  iteration time= 0.00829768180847168
max ddh =  7.31201946807e-06
step = 9941  avg d_h= 0.559606996165
Number of iterations = 1 tolerance = 1.48128297094e-05  iteration time= 0.0076444149017333984
Number of iterations = 2 tolerance = 8.18396315386e-09  iteration time= 0.007498025894165039
max ddh =  7.31136074249e-06
step = 9942  avg d_h= 0.559576132556
Number of iterations = 1 tolerance = 1.48171987697e-05  iteration time= 0.0069203376770

Number of iterations = 2 tolerance = 2.71473493685e-09  iteration time= 0.007901430130004883
max ddh =  7.2883535896e-06
step = 9977  avg d_h= 0.558497012779
Number of iterations = 1 tolerance = 1.47663302643e-05  iteration time= 0.00856781005859375
Number of iterations = 2 tolerance = 8.84762241249e-09  iteration time= 0.007833242416381836
max ddh =  7.28769777329e-06
step = 9978  avg d_h= 0.558466212384
Number of iterations = 1 tolerance = 1.47635589983e-05  iteration time= 0.008195161819458008
Number of iterations = 2 tolerance = 3.99010697833e-09  iteration time= 0.007116079330444336
max ddh =  7.28703878891e-06
step = 9979  avg d_h= 0.558435413742
Number of iterations = 1 tolerance = 1.47652415188e-05  iteration time= 0.007688045501708984
Number of iterations = 2 tolerance = 3.98817958432e-09  iteration time= 0.006865262985229492
max ddh =  7.28638086877e-06
step = 9980  avg d_h= 0.558404616854
Number of iterations = 1 tolerance = 1.47611355227e-05  iteration time= 0.0071907043457

In [6]:
subplot(2,2,1)
imshow_grid_at_node(mg, h, var_name='Head ($m$)', cmap='viridis',vmin=300,vmax=400)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        #ylim([300,400])
subplot(2,2,2)
plot_links(mg, 'conduit__discharge', magnitude=True, var_name='Discharge ($m^3/s$)', vmin=0., vmax=1, autoscale=False, pixels_per_node=500)
#plot(mg.at_link['conduit__discharge'][mg.active_links])
#ylabel('')
#ylim([0,1.5])
subplot(2,2,3)
plot_links(mg, 'hydraulic__diameter', var_name='Hydraulic diameter ($m$)', autoscale=False, vmin=0, vmax=1.25, pixels_per_node=500)
subplot(2,2,4)#, position=[0.2,0.2,0.9,0.9])
plot(mg.at_link['hydraulic__diameter'][mg.active_links])
xlabel('Link number')
ylabel('Hydraulic diameter ($m$)')
ylim(0,1.25)
#ylim([0.5,1.5])
image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
#tight_layout()
subplots_adjust(wspace=0.3)
#savefig(image_name)

image data type = [[[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]

 [[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]

 [[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]]
image data type = [[[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]

 [[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]

 [[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84

In [5]:
figure()
imshow_grid_at_node(mg, h)
title('head')

figure()
plot_links(mg, 'conduit__discharge')#,logvalues=True)
title('discharge')


figure()
plot_links(mg, 'hydraulic__diameter')#,logvalues=True)
title('d_H')


image data type = <PIL.Image.Image image mode=RGBA size=0x0 at 0x7F558C2C5B80>


TypeError: Image data of dtype object cannot be converted to float

In [7]:
mg.at_link['conduit__discharge'].dtypeype

dtype('float64')

In [5]:
figure()
plot(mg.at_link['conduit__discharge'][mg.active_links])

In [16]:
mc.run_one_step()
pfn.run_one_step()

#mc = MeltCreep(mg, dt=5000.)


max ddh =  4.545746571648965e-05
Number of iterations = 1 tolerance = 6.646520870778883e-05
Number of iterations = 2 tolerance = 8.926521188890035e-09


In [12]:
mg.calc_net_flux_at_node(Q, out=net_node_flux)
net_Q = mg.calc_net_flux_at_node(Q)/100. - mg.at_node['input__discharge'] 
figure()
imshow_grid_at_node(mg, net_Q)

NameError: name 'net_node_flux' is not defined

In [20]:
mg.at_node['input__discharge']

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [24]:
from scipy.optimize import fsolve
def network_residuals(heads):
    h[mg.core_nodes] = heads
    mg.calc_grad_at_link(h, out=dhdx)
    Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    return mg.calc_net_flux_at_node(Q, out=net_node_flux)[mg.core_nodes]/mg.dx - mg.at_node['input__discharge'][mg.core_nodes]
#This function doesn't account for inflow at nodes


In [25]:
mc.run_one_step()
mg.calc_grad_at_link(h, out=dhdx)
mg.at_node['hydraulic__head'][mg.core_nodes] = fsolve(network_residuals, h[mg.core_nodes])
mg.at_link['conduit__discharge'][mg.core_nodes] = np.sign(dhdx[mg.core_nodes])*np.sqrt(np.fabs(dhdx[mg.core_nodes]))

print "mean h =", mean(mg.at_node['hydraulic__head'][mg.core_nodes] )
print "mean Q =", mean(Q)

mean ddh =  -5.66192997218e-18
mean h = 166.222550077
mean Q = 0.0865514465643


In [22]:
mg.dx

100.0

In [26]:
figure()
imshow_grid_at_node(mg, h)

In [24]:
figure()
plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
print mean(mg.at_link['hydraulic__diameter'])

3.50074727364e+69


In [56]:
figure()
plot_links(mg, 'conduit__discharge', autoscale=True, magnitude=True)
#print mean(mg.at_link['hydraulic__diameter'])

In [10]:
mc = MeltCreep(mg, dt=5000.)
#mean(mc.Q)

In [17]:
plot(mg.at_link['hydraulic__diameter'][200:300])


Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902


Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.6482677381

In [19]:
#figure()

pfn.run_one_step()
mc.run_one_step()

Number of iterations = 1 tolerance = 0.0031818002267
Number of iterations = 2 tolerance = 1.26436621244e-05
mean ddh =  -6.86035808489e-06


In [41]:
#for m in mc.melt:
#    print m
print max(mc.melt[~np.isnan(mc.melt)])
print max(mc.creep)
#print mc.d_h.mean()
#print mg.at_link['hydraulic__diameter']

3.35041761374e-06
2.12710591355e-06


In [15]:
#mc.run_one_step()

mc._grid.at_link['conduit__discharge']

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])